In [ ]:
%matplotlib notebook   
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as ticker


df=pd.read_csv("data/C2A2_data/BinnedCsvs_d25/9b8a3493628de4a545ca348012ca3563e0a85b955446eef6f8fdafb5.csv")
df = df.copy()
df = df.sort_values(by='Date', ascending=True)
df['Data_Value'] = df['Data_Value']/10
df = df[~df['Date'].str.endswith('02-29')]
df_10 = df[~df['Date'].str.startswith('2015')]
df_10['Date'] = df_10['Date'].str[5:]
df_max = df_10[df_10['Element']=='TMAX']
df_min = df_10[df_10['Element']=='TMIN']
df_max = df_max.groupby('Date').agg({'Data_Value':np.max})
df_min = df_min.groupby('Date').agg({'Data_Value':np.min})
data = pd.merge(df_max,df_min,how='outer', left_index=True, right_index=True)
data=data.rename(columns={'Data_Value_x':'TMAX','Data_Value_y':'TMIN'})
#data is dataframe of tmax and tmin

df_2015 = df[df['Date'].str.startswith('2015')]
df_2015['Date'] = df_2015['Date'].str[5:]
df_max15 = df_2015[df_2015['Element']=='TMAX']
df_min15 = df_2015[df_2015['Element']=='TMIN']
df_max15 = df_max15.groupby('Date').agg({'Data_Value':np.max})
df_min15 = df_min15.groupby('Date').agg({'Data_Value':np.min})
data_15 = pd.merge(df_max15,df_min15,how='outer', left_index=True, right_index=True)
data_15=data_15.rename(columns={'Data_Value_x':'TMAX','Data_Value_y':'TMIN'})

data_all = pd.merge(data,data_15,how='outer', left_index=True, right_index=True)
max_break = data_all[data_all['TMAX_y']>data_all['TMAX_x']]['TMAX_y']
min_break = data_all[data_all['TMIN_y']<data_all['TMIN_x']]['TMIN_y']
max_break = max_break.reset_index(drop=False).rename(columns={'TMAX_y':'MAX_break'})
min_break = min_break.reset_index(drop=False).rename(columns={'TMIN_y':'MIN_break'})
#break values 

plt.figure()
observation_dates = np.arange('2005-01-01', '2006-01-01', dtype='datetime64[D]')
observation_dates = list(map(pd.to_datetime, observation_dates))
plt.plot(observation_dates,data['TMAX'], lw=1,c='lightcoral',alpha=0.5, label='The record high (2005-2014)')
plt.plot(observation_dates,data['TMIN'], lw=1,c='skyblue',alpha=0.8, label='The record low (2005-2014)')
plt.xlim('2005-01-01', '2006-01-01')
#ax=plt.gca()
#ax.xaxis.set_major_locator(MonthLocator())

max_break['Date'] = '2005-' + max_break['Date']
min_break['Date'] = '2005-' + min_break['Date']
plt.scatter(list(map(pd.to_datetime, max_break['Date'])),max_break['MAX_break'], s=6, c='red', label='Record-breaking high in 2015')
plt.scatter(list(map(pd.to_datetime, min_break['Date'])),min_break['MIN_break'], s=6, c='blue', label='Record-breaking low in 2015')

plt.xlabel('Date')
plt.ylabel('The degree Celsius')
plt.title('Temperature records near Happy Valley, Wan Chai, and Hong Kong (2005-2015)', fontsize=10)
plt.legend(fontsize=8,frameon=False)
plt.gca().fill_between(observation_dates, 
                       data['TMIN'], data['TMAX'], 
                       facecolor='grey', 
                       alpha=0.25)
ax=plt.gca()
ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=15))
ax.xaxis.set_major_formatter(ticker.NullFormatter())
ax.xaxis.set_minor_formatter(dates.DateFormatter('%b'))

# 1 refers to the bottom of the plot for xticks and the left for yticks
# 2 refers to the top of the plot for xticks and the right for yticks
for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0)
    #tick.tick2line.set_markersize(0)
    tick.label1.set_horizontalalignment('center')
